# Protein folding with Metropolis and Quantum Walks

Work plan

1. Program Metropolis in superposition using Qiskit. From arXiv:1910.01659
3. Figure out how to rotate the structure of the proteins. Perhaps use psi4 to figure out the energy of each configuration
4. Prove it with usual Metropolis and Tripeptides: https://en.wikipedia.org/wiki/Tripeptide

Since we are going to use IBM simulator, we can use up to 32 qubits!

Remember that to convert some subcircuit to a gate, one uses:
sub_inst = sub_circ.to_instruction()
circ.append(sub_inst, [q[1], q[2]])

In [1]:
# Import libraries
from qiskit import *
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit, Qubit, Clbit, Gate, Parameter 
from qiskit.aqua.components.oracles import Oracle, TruthTableOracle
import numpy as np
import math
from copy import deepcopy

qiskit.__qiskit_version__

import logging

from sympy.combinatorics.graycode import GrayCode
from math import pi

from qiskit.aqua.utils.controlled_circuit import apply_cu3
from qiskit.aqua import AquaError

%matplotlib inline

The first thing we need to do is to calculate the registers we need in order to be able to perform the metropolis algorithm:

For example, for a tripeptide (n=3) we have 2(n-1) angles

Coin: 
1 to allow or decline move

Move: 2 moves on 4 angles:
2 to identify angle
1 to identify if we increase or decrease it

State: 32 divisions (5 qubits) for each angle
20 qubits = 4 angles * 5 qubits/angle

Ancillas: 
8 qubits

Total: 32 qubits, maximum

In [2]:
#Indicate number of precision bits
n_precision_bits = 5

n_ancilla_bits = 10 #For the oracle

In [3]:
# For tripeptide
'''
# State definition. 32 divisions for each of the four angles. All angles range from 0 to 2pi
angle_0 = QuantumRegister(5, name = 'angle_0')
angle_1 = QuantumRegister(5, name = 'angle_1') 
angle_2 = QuantumRegister(5, name = 'angle_2') 
angle_3 = QuantumRegister(5, name = 'angle_3') 


# Move proposal
move_id = QuantumRegister(2, name = 'move_id') #Which angle are we modifying
move_value = QuantumRegister(1, name = 'move_value') #0 -> decrease the angle. 1-> increase it

# Coin
coin = QuantumRegister(1, name = 'coin')

# Ancillas
ancilla = QuantumRegister(8, name = 'ancilla')



# Circuit
qc = QuantumCircuit(angle_0,angle_1,angle_2,angle_3,move_id,move_value,coin,ancilla)
'''

# For dipeptide
# State definition. 32 divisions for each of the four angles. All angles range from 0 to 2pi
angle_phi = QuantumRegister(n_precision_bits, name = 'angle_phi')
angle_psi = QuantumRegister(n_precision_bits, name = 'angle_psi') 

# Move proposal
move_id = QuantumRegister(1, name = 'move_id') #Which angle are we modifying
move_value = QuantumRegister(1, name = 'move_value') #0 -> decrease the angle. 1-> increase it

# Coin
coin = QuantumRegister(1, name = 'coin')

# Ancillas
ancilla = QuantumRegister(n_ancilla_bits, name = 'ancilla')


# Circuit
qc = QuantumCircuit(angle_phi,angle_psi,move_id,move_value,coin,ancilla)

Next we have to define the steps of the Metropolis using Quantum Walk

In [4]:
# This is the move preparation gate
def move_preparation(circuit,move_id,move_value):
    '''
    Proposes new moves
    '''
    circuit.h(move_id)
    circuit.h(move_value)

# Let us define it as a portable gate
s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)

sub_circ = QuantumCircuit(s_move_id, s_move_value)

move_preparation(sub_circ,s_move_id,s_move_value)

move_preparation = sub_circ.to_instruction()


# Use as qc.append(move_preparation, [move_id[0], move_value[0]])

Next, we define the conditional move. For that we have to go over all possible combinations of move_id and move_value, adding or substracting 1, conditioned on the coin being at state 1

In [5]:
def conditional_move_tripeptide(circuit,coin,move_id,move_value,angle_id,angle_value,ancilla):
    '''
    Conditioned on coin, perform a move. For a tripeptide!
    We use a repetitive structure where we perform the conditional sum and substraction for each angle.
    '''
    # For angle_0 ----------------------------------------------
    circuit.x(move_id) # Put move_id in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])
    #circuit.append(Gate("mcx", 5, []), [move_id[0], move_id[1], move_value, coin, ancilla[0]]) #create a single control
    sum1(circuit,angle_0,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_0,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id)
    
    # For angle_1 ----------------------------------------------
    circuit.x(move_id[1]) # Put move_id in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_1,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_1,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id[1])
    
    # For angle_2 ----------------------------------------------
    circuit.x(move_id[0]) # Put move_id in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_2,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_2,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id[0])
    
    # For angle_3 ----------------------------------------------
    #circuit.x(move_id[0]) #  move_id  is in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_3,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_3,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) = 6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    #circuit.x(move_id[0])
    
# Let us define it as a portable gate
s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)
s_coin = QuantumRegister(coin.size)
s_ancilla = QuantumRegister(ancilla.size)
s_angle_0 = QuantumRegister(5, name = 'angle_0')
s_angle_1 = QuantumRegister(5, name = 'angle_1') 
s_angle_2 = QuantumRegister(5, name = 'angle_2') 
s_angle_3 = QuantumRegister(5, name = 'angle_3') 

sub_circ = QuantumCircuit(s_angle_0,s_angle_1,s_angle_2,s_angle_3,s_move_id,s_move_value,s_coin,s_ancilla)
conditional_move_tripeptide(sub_circ,s_coin,s_move_id,s_move_value,s_angle_id,s_angle_value,s_ancilla)
conditional_move = sub_circ.to_instruction()

qc.append(conditional_move.inverse(), [(angle_0[j] for j in range(4)),(angle_1[j] for j in range(4)),(angle_2[j] for j in range(4)),(angle_3[j] for j in range(4)),move_id[0], move_id[1], move_value[0],coin[0],ancilla[0],ancilla[1],ancilla[2]])

NameError: name 's_angle_id' is not defined

In [6]:
def conditional_move_dipeptide(circuit,coin,move_id,move_value,angle_phi,angle_psi,ancilla):
    '''
    Conditioned on coin, perform a move. For a dipeptide!
    We use a repetitive structure where we perform the conditional sum and substraction for each angle.
    Checked ok
    '''
    # For angle_phi = angle_id = 0 ----------------------------------------------
    circuit.x(move_id) # Put move_id in 1
    
    # Conditional on move_id = 0, move_value = 1 and coin = 1, increase angle_phi by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])
    #circuit.append(Gate("mcx", 5, []), [move_id[0], move_id[1], move_value, coin, ancilla[0]]) #create a single control
    sum1(circuit,angle_phi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])
    
    # Conditional on move_id = 0, move_value = 0 and coin = 1, decrease angle_ psi by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_phi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id)
    
    # For angle_psi = angle_id = 1 ----------------------------------------------
    # Put move_id in 11
    
    # Conditional on move_id = 1, move_value = 1 and coin = 1, increase angle_psi by one
    circuit.mcrx(theta = pi, q_controls = [move_id[0],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_psi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 1, move_value = 0 and coin = 1, decrease angle_psi by one
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_psi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    
# Let us define it as a portable gate
'''s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)
s_coin = QuantumRegister(coin.size)
s_ancilla = QuantumRegister(ancilla.size)
s_angle_phi = QuantumRegister(precision_bits, name = 'angle_phi')
s_angle_psi = QuantumRegister(precision_bits, name = 'angle_psi') 


sub_circ = QuantumCircuit(s_angle_phi,s_angle_psi,s_move_id,s_move_value,s_coin,s_ancilla)
conditional_move_dipeptide(sub_circ,s_coin,s_move_id,s_move_value,s_angle_phi,s_angle_psi,s_ancilla)
conditional_move = sub_circ.to_instruction()
'''
# To use it or its inverse
# qc.append(conditional_move.inverse(), [(angle_phi[j] for j in range(precision_bits)),(angle_psi[j] for j in range(precision_bits)),move_id[0], move_value[0], coin[0], ancilla[0], ancilla[1], ancilla[2]])

# Validation circuit. ------------------------------------------
coin = QuantumRegister(1)
move_id = QuantumRegister(1)
move_value = QuantumRegister(1)
angle_phi = QuantumRegister(3)
angle_psi = QuantumRegister(3)
ancilla = QuantumRegister(3)

ccoin = ClassicalRegister(1)
cmove_id = ClassicalRegister(1)
cmove_value = ClassicalRegister(1)
cangle_phi = ClassicalRegister(3)
cangle_psi = ClassicalRegister(3)
cancilla = ClassicalRegister(3)

qc = QuantumCircuit(coin,move_id, move_value,angle_phi, angle_psi,ancilla, ccoin, cmove_id, cmove_value, cangle_phi, cangle_psi, cancilla)

# Initialize state
qc.x(coin)
qc.x(angle_phi[0])
#qc.x(qubit_string)

# Gate
conditional_move_dipeptide(qc,coin,move_id,move_value,angle_phi,angle_psi,ancilla)

# Readout
#Measure
qc.measure(coin,ccoin)
qc.measure(move_id,cmove_id)
qc.measure(move_value,cmove_value)
qc.measure(angle_phi, cangle_phi)
qc.measure(angle_psi, cangle_psi)
qc.measure(ancilla,cancilla)

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(qc, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(qc)

counts

NameError: name 'sum1' is not defined

In [14]:
def reflection(circuit,move_id,move_value,coin):
    '''
    I have to investigate over what is the reflection performed. Is it performed over 000?
    If in state 0000, make it 1111, cccz gate and back to 0000
    '''
    circuit.x(move_id)
    circuit.x(move_value)
    circuit.x(coin)
    
    circuit.mcrz(lam = pi, q_controls = [move_id[0]] + [move_value[0]], q_target = coin[0]) #For dipeptide
    # circuit.mcrz(lam = pi, q_controls = [move_id[0]] + [move_id[1]]+ [move_value[0]], q_target = coin[0]) #For tripeptide
    
    circuit.x(move_id)
    circuit.x(move_value)
    circuit.x(coin)
    

# Let us define it as a portable gate
s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)
s_coin = QuantumRegister(coin.size)


sub_circ = QuantumCircuit(s_move_id,s_move_value,s_coin)
reflection(sub_circ,s_move_id,s_move_value,s_coin)
reflection = sub_circ.to_instruction()

# Use as qc.append(reflection.inverse(), [move_id[0], move_value[0],coin[0]])

We now define the coin flip, which takes three steps: 
1. Calculating the oracle that outputs the probability encoded in the ancilla
2. Performing the coin flip
3. Undoing the oracle to uncompute the ancilla

We create a subcircuit sub_circ with registers of type 's_ ' where we perform all the calculations.
Unfortunately, we will have to go over all this next lines in order to perform the entire calculation, which cannot be packed in a gate due to the dependence to the beta parameter.

Notice that the main difficulty is building a parametrised instruction.

In [113]:
def coin_flip(circuit,coin,ancilla):
    '''
    Prepares the coin with the probability encoded in the ancilla, or unprepares it if direct == -1
    '''
    
    #Necesitamos usar el número guardado en las ancillas para realizar rotaciones controladas.  
    #Notice that ancilla encodes 1-probability, rather than probability.
    #Notice also that cu3(theta) rotates theta/2. As the first angle to rotate is pi/4 we need to start in theta = pi/2

    circuit.x(coin) # Start in 1 and decrease it, since we encoded the angle corresponding 1-probability
    for i in range(ancilla.size):
        circuit.cu3(theta = -math.pi/(2**(i+1)), phi  = 0, lam = 0, control_qubit = ancilla[i], target_qubit = coin)     
        
#beta = Parameter('β')

#oracle_circuit = beta_precalc_TruthTableOracle(energies_dictionary,beta,out_bits = n_ancilla_bits)
def coin_flip_func(oracle_circuit):
    oracle_gate = oracle_circuit.to_instruction() 

    # Let us create a circuit for coin_flip
    s_move_id = QuantumRegister(move_id.size) 
    s_move_value = QuantumRegister(move_value.size)
    s_coin = QuantumRegister(coin.size)
    s_ancilla = QuantumRegister(ancilla.size)
    s_angle_phi = QuantumRegister(precision_bits, name = 'angle_phi')
    s_angle_psi = QuantumRegister(precision_bits, name = 'angle_psi') 

    [s_angle_phi, s_angle_psi, s_move_id, s_move_value] = oracle.variable_register 
    s_ancilla = oracle.output_register 
    sub_circ = QuantumCircuit(oracle.variable_register,s_coin,s_ancilla)

    #Main operations on the subcircuit

    sub_circ.append(oracle_gate, [oracle.variable_register, s_ancilla])
    coin_flip(sub_circ,s_coin,s_ancilla)
    sub_circ.append(oracle_gate.inverse(), [oracle.variable_register, s_ancilla])

    #Create a general beta-parametrised gate for whole circuit
    coin_flip_gate = sub_circ.to_instruction()

'''
Use as:
    coin_flip_gate.params[0]= a_given_beta
    qc.append(coin_flip_gate.inverse(), [(angle_psi[j] for j in range(angle_psi.size)),angle_phi[j] for j in range(angle_phi.size)),move_id[0], move_value[0],coin[0],(ancilla[j] for j in range(ancilla.size))])
'''

TypeError: '>' not supported between instances of 'ParameterExpression' and 'float'

Next thing we should do is create the entire loop.

# The loop is given by

Still need the energies_dictionary.
The main loop is $W = RV^\dagger B^\dagger FBV$

In [ ]:
def W_func(oracle_circuit):

    # State definition. All angles range from 0 to 2pi
    angle_phi = QuantumRegister(precision_bits, name = 'angle_phi')
    angle_psi = QuantumRegister(precision_bits, name = 'angle_psi') 

    # Move proposal
    move_id = QuantumRegister(1, name = 'move_id') #Which angle are we modifying
    move_value = QuantumRegister(1, name = 'move_value') #0 -> decrease the angle. 1-> increase it

    # Coin
    coin = QuantumRegister(1, name = 'coin')

    # Ancillas
    ancilla = QuantumRegister(ancilla_bits, name = 'ancilla')


    # Circuit
    qc = QuantumCircuit(angle_phi,angle_psi,move_id,move_value,coin,ancilla)


    # Move preparation
    qc.append(move_preparation, [move_id[0], move_value[0]])

    # Coin flip
    #D beta = Parameter('β')
    #D coin_flip_gate.params[0]= beta
    coin_flip_func(oracle_circuit)
    qc.append(coin_flip_gate, [(angle_phi[j] for j in range(angle_phi.size)),(angle_psi[j] for j in range(angle_psi.size)),move_id[0], move_value[0],coin[0],(ancilla[j] for j in range(ancilla.size))])

    # Conditional move
    qc.append(conditional_move, [(angle_phi[j] for j in range(precision_bits)),(angle_psi[j] for j in range(precision_bits)),move_id[0], move_value[0], coin[0], ancilla[0], ancilla[1], ancilla[2]])

    # Inverse coin flip
    qc.append(coin_flip_gate.inverse(), [(angle_phi[j] for j in range(angle_phi.size)),(angle_psi[j] for j in range(angle_psi.size)),move_id[0], move_value[0],coin[0],(ancilla[j] for j in range(ancilla.size))])

    # Inverse move preparation
    qc.append(move_preparation, [move_id[0], move_value[0]])

    # Reflection
    qc.append(reflection, [move_id[0], move_value[0],coin[0]])

    W_gate = qc.to_instruction()

# This defines the parametrised gate W, and we can reuse its inverse too.

'''
Use as:
    #W_gate.params[0]= a_given_beta
    qc.append(W_gate.inverse(), [(angle_psi[j] for j in range(angle_psi.size)),angle_phi[j] for j in range(angle_phi.size)),move_id[0], move_value[0],coin[0],(ancilla[j] for j in range(ancilla.size))])
'''

# General circuit

In [ ]:
# State definition. All angles range from 0 to 2pi
g_angle_phi = QuantumRegister(precision_bits, name = 'angle_phi')
g_angle_psi = QuantumRegister(precision_bits, name = 'angle_psi') 

# Move proposal
g_move_id = QuantumRegister(1, name = 'move_id') #Which angle are we modifying
g_move_value = QuantumRegister(1, name = 'move_value') #0 -> decrease the angle. 1-> increase it

# Coin
g_coin = QuantumRegister(1, name = 'coin')

# Ancillas
g_ancilla = QuantumRegister(ancilla_bits, name = 'ancilla')

# Circuit
circuit = QuantumCircuit(g_angle_phi,g_angle_psi,g_move_id,g_move_value,g_coin,g_ancilla)

# Define number of steps
L=1000
beta_max = 1000
# Read energies_dictionary

# Metropolis algorithm
#lista = []
for i in range(L):
    beta = i/L*beta_max
    ''' HERE WE INPUT THE ENERGIES_DICTIONARY'''
    oracle_circuit = beta_precalc_TruthTableOracle(energies_dictionary,beta,out_bits = n_ancilla_bits)
    
    W_func(oracle_circuit)
    
    #lista.append(deepcopy(W_gate)) # We deepcopy W_gate to not interfere with other calls
    #lista[i].params[0]= beta
    circuit.append(W_gate, [(g_angle_phi[j] for j in range(g_angle_phi.size)),(g_angle_psi[j] for j in range(g_angle_psi.size)),g_move_id[0], g_move_value[0],g_coin[0],(g_ancilla[j] for j in range(g_ancilla.size))])


''' #Old version
for i in range(L):
    W_gate.params[0] = i/L*beta_max  # Warning!! Does it change the beta for all W already in the circuit? If so, deepcopy?
    circuit.append(W_gate, [(g_angle_phi[j] for j in range(g_angle_phi.size)),(g_angle_psi[j] for j in range(g_angle_psi.size)),g_move_id[0], g_move_value[0],g_coin[0],(g_ancilla[j] for j in range(g_ancilla.size))])
'''
    
# Add measurements
circuit.barrier(range(32))


phi = ClassicalRegister(g_angle_phi.size)
psi = ClassicalRegister(g_angle_psi.size)
circuit += QuantumCircuit(phi,psi)

# map the quantum measurement to the classical bits
circuit.measure(g_angle_phi,phi)
circuit.measure(g_angle_psi,psi)

# Execute the circuit
backend = BasicAer.get_backend('qasm_simulator')
job = execute(circuit, backend, shots=1024) # Start with that and move to 4096
job.result().get_counts(circuit)

# Helper functions

In [5]:
def sum1(circuit,qubit_string,control,start,end):
    '''
    Outputs:
    Sums register 1 and 2 (1 qubit) in register 1. Tested ok.
    
    Input:
    circuit: QuantumCircuit with registers qubit_string, control, ancilla
    
    qubit_string: QuantumRegister
    
    control: Qubit. Use ancilla[0] or similar
    
    start: Qubit. Use ancilla[1] or similar
    end: Qubit. Use ancilla[2] or similar
    '''
    n_qubits = qubit_string.size     # calculate n_qubits
    circuit.cx(control,end) # iff control = 1, end = 1
    circuit.x(start)
    circuit.cx(control,start) # iff control = 1, start = 0
    
    for i in range(n_qubits+1): #Don't need to add control, since start already does that work
        '''
        Next thing we analise if all qubits to the right have value 1, 
        and save it in the current qubit and start
        '''
        if i > 0:
            # For i = 0, there is only the start to worry about
            circuit.mcrx(theta = pi, q_controls = [qubit_string[j] for j in range(n_qubits-i)]+[end], q_target = qubit_string[n_qubits-i])
        circuit.mcrx(theta = pi, q_controls = [qubit_string[j] for j in range(n_qubits-i)]+[end], q_target = start)

        '''
        Next, controlling on the current qubit and start, we change all the following qubits to 0.
        We have to control with qubit_string[n_qubit]
        '''
        if i == 0:
            for j in range(n_qubits-i):
                circuit.ccx(control,start,qubit_string[j])
            circuit.ccx(control,start,end)
        elif i == n_qubits:
            circuit.mcrx(theta = pi, q_controls = [control,qubit_string[n_qubits-i],start], q_target = end)
        else:
            for j in range(n_qubits-i):            
                circuit.mcrx(theta = pi, q_controls = [control,qubit_string[n_qubits-i],start], q_target = qubit_string[j])
            circuit.mcrx(theta = pi, q_controls = [control,qubit_string[n_qubits-i],start], q_target = end)
    circuit.x(start)
    
    
# Validation circuit. ------------------------------------------
qubit_string = QuantumRegister(4)
#control = QuantumRegister(1)
ancilla = QuantumRegister(3)

bit_string = ClassicalRegister(4)
#ccontrol = ClassicalRegister(1)
cancilla = ClassicalRegister(3)

qc = QuantumCircuit(qubit_string, ancilla, bit_string, cancilla)

# Initialize state
qc.x(ancilla[0])
#qc.x(qubit_string)

# Gate
sum1(qc,qubit_string,ancilla[0],ancilla[1],ancilla[2])

# Readout
#Measure
qc.measure(qubit_string,bit_string)

qc.measure(ancilla[0],cancilla[0])
qc.measure(ancilla[1],cancilla[1])
qc.measure(ancilla[2],cancilla[2])

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(qc, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(qc)

counts

{'001 0001': 1024}

In [6]:
def substract1(circuit,qubit_string,control,start,end):
    '''
    Outputs:
    Substracts register 2 (1 qubit) from register 1 in register 1. Tested ok.
    
    Input:
    circuit: QuantumCircuit with registers qubit_string, control, ancilla
    
    qubit_string: QuantumRegister
    
    control: Qubit. Use ancilla[0] or similar
    
    start: Qubit. Use ancilla[1] or similar
    end: Qubit. Use ancilla[2] or similar
    
    Comments: In binary, substracting is the same procedure as summing when we exchange 0s and 1s
    '''
    circuit.x(qubit_string)

    sum1(circuit,qubit_string,control,start,end)
    
    circuit.x(qubit_string)
    

# Validation circuit. ------------------------------------------
qubit_string = QuantumRegister(4)
#control = QuantumRegister(1)
ancilla = QuantumRegister(3)

bit_string = ClassicalRegister(4)
#ccontrol = ClassicalRegister(1)
cancilla = ClassicalRegister(3)

qc = QuantumCircuit(qubit_string, ancilla, bit_string, cancilla)

# Initialize state
qc.x(ancilla[0])
#qc.x(qubit_string)

# Gate
substract1(qc,qubit_string,ancilla[0],ancilla[1],ancilla[2])

# Readout
#Measure
qc.measure(qubit_string,bit_string)
qc.measure(ancilla[0],cancilla[0])
qc.measure(ancilla[1],cancilla[1])
qc.measure(ancilla[2],cancilla[2])

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(qc, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(qc)

counts

{'001 1111': 1024}

In [7]:
def exp(x):
    result = 1
    denominator = 1
    power = x
    for i in range(1,20):
        result += power/denominator
        power *= x
        denominator *= (i+1)
    return result

print(exp(-5))
print(np.exp(-5))

def arcsin_sqrt(x):
    y = x-1/2
    result = math.pi/4
    denominator = 1

    power = y
    power_times = 1
    numerator = 1

    for i in range(0,20):
        numerator = math.factorial(2*i)/(math.factorial(i)*math.factorial(i))
        result  += (numerator/denominator)*power
        
        power *= y *y
        power_times +=2
        denominator += 2

    return result

print(arcsin_sqrt(.9))
print(np.arcsin(np.sqrt(.9)))


def int_angle_func(angle,out_bits):
    out_str = ''
    a = angle
    for bits in range(1,out_bits+1):

        if a +1e-10 > 1/(2**(bits)):
            out_str += '1'
            a -= 1/(2**bits)
        else:
            out_str += '0'
    
    return out_str


angle = .0
out_bits = 8
int_angle = format(int(angle*2**out_bits), 'b')
print(int_angle)
boolean = int_angle_func(angle,out_bits)
print(boolean)

0.00670634105421557
0.006737946999085467
1.2490453676434239
1.2490457723982544
0
00000000


In [8]:
class beta_precalc_TruthTableOracle(TruthTableOracle):
    '''Outputs the binary angle of rotation to get the correct probability. Tested ok'''
    def __init__(self, energies_dictionary, beta, out_bits, optimization=True, mct_mode='basic'):
        self.beta = beta
        self.out_bits = out_bits
        self.in_bits = len(list(energies_dictionary.keys())[0]) # The key of energies_dictionary is the input bits
        self.energies_dictionary = energies_dictionary
        print('Is parameter',self.parameters)
        self.calculate_bitmap()
        super().__init__(self.bitmap, optimization, mct_mode)
        
    def calculate_bitmap(self):
        new_bitmap = []
        angles = {}
        for i in range(int(2**self.in_bits)):
            st = '0'*(self.in_bits - len(str(format(i,'b')))) + str(format(i,'b'))
            #print(st)
            if self.energies_dictionary[st] >= 0:
                #print(type(np))
                probability = math.exp(-self.beta * self.energies_dictionary[st])
                #print('probability: ',probability)
            else: 
                probability = 1
                
            # Instead of encoding the probability, we will encode 1-probability. That way 1 -> 000, 
            #but if probability is 0 there is some small probability of acceptance
            probability = 1 - probability
            
            
            # Instead of probability save angles so rotations are easier to perform afterwards sqrt(p) = sin(theta)
            angle = math.asin(math.sqrt(probability))
            
            # Make the angle be between [0,1]. Since the maximum is pi/2
            angle /= (math.pi/2) 
            
            # Convert it into an integer and a string
            #int_angle = format(int(angle*2**self.out_bits), 'b')
            #str_angle = str(int_angle) 
            str_angle2 = int_angle_func(angle,out_bits)
            # Convert it to binary
            #int_angle = format(int(angle*2**out_bits), 'b')
            '''
            if int_angle == '1' + '0'*out_bits:
                angles[st] = '1'*out_bits # As we only have out_bits, the 10000 is substituted by 1111
            else:
                str_angle = str(int_angle)
                angles[st] = '0'*(out_bits - len(str_angle)) + str_angle
            ''' 
            angles[st] = str_angle2
            #print('dict_key',st)
            #print('energies',self.energies_dictionary[st])
            #print('probability',probability)
            #print('angle',angle)
            #print('angles', angles[st])
        
        # Encoding the new bitmap
        new_bitmap = []
        for o in range(self.out_bits):
            string = ''
            for i in range(int(2**self.in_bits)):
                st = '0'*(self.in_bits - len(str(format(i,'b')))) + str(format(i,'b'))
                string += str(angles[st])[o]
            new_bitmap += [string]
        print('angles',angles)
        print('bitmap',new_bitmap)
        self.bitmap = new_bitmap
        



In [109]:
'''# Validation -----------------
energies_dictionary = {}
beta = 1

# Lets set 2 bits for phi and 2 for phi. Lets create a dictionary of random energies.
energies = {}
for i in range(16):
    key = str(format(i, 'b'))
    key = '0'*(4 - len(key)) + key
    energies[key] = np.random.random()
    '''
#print(energies)
    
for key in energies.keys():
    for angle_select in ('0','1'):
        for plus_minus in ('0','1'):
            
            phi = int(key[:2])
            psi = int(key[2:])
            angle_s = int(angle_select)
            p_s = 2*int(plus_minus)-1
            
            phi_n = (phi + p_s*(1-angle_s) + 4) % 4 # The (x +4 )%4 ensures the number is positive
            psi_n = (psi + p_s*(angle_s) + 4) % 4
            phi_s = str(format(int(phi_n),'b'))
            psi_s = str(format(int(psi_n),'b'))
            phi_s = '0'*(2 - len(phi_s)) + phi_s
            psi_s = '0'*(2 - len(psi_s)) + psi_s
            
            new_key = phi_s + psi_s
            
            e_old = energies[key]
            e_new = energies[new_key]
            
            energies_dictionary[key + angle_select + plus_minus] = e_new - e_old

            
#print('energies_dictionary')
print(energies_dictionary)            
            
out_bits = 6            

oracle = beta_precalc_TruthTableOracle(energies_dictionary, beta, out_bits)

print('bitmap')

print(oracle.bitmap)

{'000000': 0.022058982295694696, '000001': -0.9255536379377155, '000010': -0.4635966389161086, '000011': -0.5685317452721831, '000100': -0.3115406616562387, '000101': -0.13340403402145917, '000110': 0.5685317452721831, '000111': -0.19193992256433878, '001000': 0.6196898373182517, '001001': 0.4302872944375923, '001010': 0.19193992256433878, '001011': 0.2968750289204133, '001100': -0.3641348102292873, '001101': 0.026470393539877013, '001110': -0.2968750289204133, '001111': 0.4635966389161086, '010000': 0.9255536379377155, '010001': 0.0285025983492756, '010010': 0.48842739256148393, '010011': 0.22361785864407324, '010100': 0.13340403402145917, '010101': 0.5676894051967489, '010110': -0.22361785864407324, '010111': 0.37175140589471267, '011000': -0.4302872944375923, '011001': 0.20863467148789439, '011010': -0.37175140589471267, '011011': -0.10694187197730198, '011100': -0.026470393539877013, '011101': 0.3337804290236832, '011110': 0.10694187197730198, '011111': -0.48842739256148393, '10000

In [19]:
np.arcsin(np.sqrt(1-np.exp(-.595895)))/(np.pi/2)

0.46743158171648164

# Experiments without relevance

Let us create a setup to check that all gates work well

In [ ]:
# First create the circuit




In [ ]:
# Apply the gate

In [ ]:
# Readout circuit


#Measure
circuit.measure(qubit_string,cqubit_string)
circuit.measure(control,ccontrol)
circuit.measure(start,cstart)
circuit.measure(end,cend)


# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(circuit, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(circuit)

counts

In [25]:
def sum1(circuit,qubit_string,control,start,end):
    '''
    Outputs:
    Sums register 1 and 2 (1 qubit) in register 1. Tested ok.
    
    Input:
    circuit: QuantumCircuit with registers qubit_string, control, ancilla
    
    qubit_string: QuantumRegister
    control: QuantumRegister with a single qubit
    
    start: Qubit. Use ancilla[0] or similar
    end: Qubit. Use ancilla[1] or similar
    '''
    n_qubits = qubit_string.size     # calculate n_qubits
    circuit.cx(control,end) # iff control = 1, end = 1
    circuit.x(start)
    circuit.cx(control,start) # iff control = 1, start = 0
    
    for i in range(n_qubits+1): #Don't need to add control, since start already does that work
        '''
        Next thing we analise if all qubits to the right have value 1, 
        and save it in the current qubit and start
        '''
        if i > 0:
            # For i = 0, there is only the start to worry about
            circuit.mcrx(theta = pi, q_controls = [qubit_string[j] for j in range(n_qubits-i)]+[end], q_target = qubit_string[n_qubits-i])
        circuit.mcrx(theta = pi, q_controls = [qubit_string[j] for j in range(n_qubits-i)]+[end], q_target = start)

        '''
        Next, controlling on the current qubit and start, we change all the following qubits to 0.
        We have to control with qubit_string[n_qubit]
        '''
        if i == 0:
            for j in range(n_qubits-i):
                circuit.ccx(control,start,qubit_string[j])
            circuit.ccx(control,start,end)
        elif i == n_qubits:
            circuit.mcrx(theta = pi, q_controls = [control[0],qubit_string[n_qubits-i],start], q_target = end)
        else:
            for j in range(n_qubits-i):            
                circuit.mcrx(theta = pi, q_controls = [control[0],qubit_string[n_qubits-i],start], q_target = qubit_string[j])
            circuit.mcrx(theta = pi, q_controls = [control[0],qubit_string[n_qubits-i],start], q_target = end)
    circuit.x(start)
    



n_qubits = 3
qubit_string = QuantumRegister(n_qubits,'string')
print(type(qubit_string))
control = QuantumRegister(1,'control')
ancilla = QuantumRegister(2, 'ancilla')
#start = ancilla[0]
#end = ancilla[1]

cqubit_string = ClassicalRegister(n_qubits,'cstring')
ccontrol = ClassicalRegister(1,'ccontrol')
cancilla = QuantumRegister(2, 'cancilla')
cstart = ClassicalRegister(1,'cstart')
cend = ClassicalRegister(1,'cend')

circuit = QuantumCircuit(qubit_string,control,ancilla,cqubit_string,ccontrol,cstart,cend)

#Choose state
circuit.x(control)
circuit.x(qubit_string)

#Sum1
sum1(circuit,qubit_string,control,ancilla[0],ancilla[1])

#Measure
circuit.measure(qubit_string,cqubit_string)
circuit.measure(control,ccontrol)
circuit.measure(ancilla[0],cstart)
circuit.measure(ancilla[1],cend)

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(circuit, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(circuit)

counts

<class 'qiskit.circuit.quantumregister.QuantumRegister'>


{'0 0 1 000': 1024}

In [27]:
ancilla = QuantumRegister(6,'a')
ancilla2 = QuantumRegister(2,'a2')
coin = QuantumRegister(1,'c')

c_ancilla = ClassicalRegister(6,'ca')
c_ancilla2 = ClassicalRegister(2,'ca2')
c_coin = ClassicalRegister(1,'cc')

circ = QuantumCircuit(ancilla,ancilla2,coin,c_ancilla,c_ancilla2,c_coin)

#--- Preparación de la ancilla

'''circ.x(ancilla)
circ.x(ancilla[3])
circ.x(ancilla[5])'''
print(len(ancilla2))
print(type([(ancilla2[i] for i in range(len(ancilla2)))]))
circ.mcrx(np.pi, [ancilla[0]] + [ancilla2[0]], q_target = coin[0])
circ.mcrx(np.pi, [ancilla[0]] + ([ancilla2[i]] for i in range(len(ancilla2))), q_target = coin[0])

#--- 

circ.barrier(ancilla,coin)
#coin_flip(circ,coin,ancilla)
circ.barrier(ancilla,coin)
circ.measure(ancilla,c_ancilla)
circ.measure(coin,c_coin)

print(circ)



2
<class 'list'>


TypeError: can only concatenate list (not "generator") to list

In [102]:
# Import Aer
from qiskit import Aer

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(circ, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(circ)
print(counts)


{'0 101010': 264, '1 101010': 760}


In [97]:
probability = 0.25
out_bits = 6

# Instead of encoding the probability, we will encode 1-probability. That way 1 -> 000, 
#but if probability is 0 there is some small probability of acceptance
#probability = 1 - probability


# Instead of probability save angles so rotations are easier to perform afterwards
angle = np.arcsin(np.sqrt(probability))

print('original angle: ',angle)
print((np.sin(angle))**2)

# Make the angle be between [0,1]. Since the maximum is pi/2
angle /= (np.pi/2) 
#print(np.pi/2)
print('formated angle',angle)

# The self.out_bits-1 appears because we reserve the 10000 for probability = 1
int_angle = format(int(angle*2**out_bits), 'b')
if int_angle == '1' + '0'*out_bits:
    str_angle = '1'*out_bits
else:
    str_angle = str(int_angle)
print('angle recorded: ',int(str_angle,2),'/',2**out_bits)

print(str_angle)
print('0'*(out_bits - len(str_angle)) + str_angle)
print((np.sin(int(str_angle,2)/2**(out_bits)*np.pi/2))**2)

original angle:  0.5235987755982988
0.24999999999999994
formated angle 0.3333333333333333
angle recorded:  21 / 64
10101
010101
21
0.24294862790338914


In [68]:
out_bits = 4
probability = 3/4
angle = np.arcsin(np.sqrt(probability))
angle /= (np.pi/2)
angle = format(int(angle*2**out_bits), 'b')
print(angle)

1010


In [ ]:
# Coin_flip: coin preparation

beta = Parameter('β')
#energies_dictionary= {}
#beta = Parameter('beta')
oracle = beta_precalc_TruthTableOracle(energies_dictionary,beta,out_bits = n_ancilla_bits)

#oracle_gate = oracle_circuit.to_instruction()


# Let us define it as a portable gate
s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)
s_coin = QuantumRegister(coin.size)
s_ancilla = QuantumRegister(ancilla.size)
s_angle_phi = QuantumRegister(precision_bits, name = 'angle_phi')
s_angle_psi = QuantumRegister(precision_bits, name = 'angle_psi') 



[s_angle_phi, s_angle_psi, s_move_id, s_move_value] = oracle.variable_register 
s_ancilla = oracle.output_register 
sub_circ = QuantumCircuit(oracle.variable_register,s_coin,s_ancilla)


oracle.construct_circuit()
oracle_circuit = oracle.circuit
oracle_gate = oracle_circuit.to_instruction()

sub_circ += oracle_circuit
coin_flip(sub_circ,s_coin,s_ancilla)
sub_circ.append(oracle_gate.inverse(), [oracle.variable_register, s_ancilla])

coin_flip_gate = sub_circ.to_instruction()
print(coin_flip_gate.params)

In [45]:
from qiskit.circuit import Parameter
from copy import deepcopy

theta = Parameter('θ')

n = 5
qubit = QuantumRegister(5)
qc = QuantumCircuit(qubit)

qc.h(0)
for i in range(n-1):
    qc.cx(i, i+1)

qc.barrier()
qc.rz(theta, range(5))
qc.barrier()

for i in reversed(range(n-1)):
    qc.cx(i, i+1)
qc.h(0)

sub_inst = qc.to_instruction()

q_bits = QuantumRegister(5)
circuit = QuantumCircuit(q_bits)

lista = []
for i in range(2):
    lista.append(deepcopy(sub_inst))
    lista[i].params[0]= np.pi/(i+1)
    circuit.append(lista[i], (q_bits[j] for j in range(q_bits.size)))

In [46]:
circuit.draw()

┌────────────────┐┌──────────────────┐
q56_0: |0>┤0               ├┤0                 ├
          │                ││                  │
q56_1: |0>┤1               ├┤1                 ├
          │                ││                  │
q56_2: |0>┤2 Circuit84(pi) ├┤2 Circuit84(pi/2) ├
          │                ││                  │
q56_3: |0>┤3               ├┤3                 ├
          │                ││                  │
q56_4: |0>┤4               ├┤4                 ├
          └────────────────┘└──────────────────┘

La conclusión es que si modificas sub_inst.params se modifican todas las aplicaciones de sub_inst